In [1]:
from IPython.display import display, Markdown, Latex
display(Markdown('*some markdown* $\phi$'))

*some markdown* $\phi$

# (Linearly) Optimising Fantasy Premier League Teams 

I've been messing around with forecasting and optimisation tools for the fantasy premier league game for a few years. It's a really rich problem and I've more than once transferred tools from FPL to real-world projects.

Being good at fantasy football may seem to be all about accurately predicting player performances, however even with perfect forecasts, the problem of selecting the optimal team is not easy -- mostly you decide on a bunch of players, shuffle them in and out of the team until you get the budget right. Introducing strategies involving transfers over several weeks adds another layer of complexity.

In this post, I will show how to optimally select an FPL team given perfect player score forecasts. Later posts will address transfer strategies, ways to correctly deal with forecast uncertainties, how to construct useful forecasts and how they interact with team optimisation.

In this post I'm going to show you how to select optimal Fantasy Premier League teams using a type of simple constrained optimisation called a linear program. I first saw someone try this in a Reddit post a few years ago, so credit to whoever wrote it.

## Linear Programming
A linear program allows the efficient exact optimisation of a objective function subject to constraints, **where the objective function and the constraints are linear in the decision variables**. This is a really broad class of problems, and these things are used all over the place.

The beautiful thing about an LP is that once you've managed to write down a linear objective and constraints, you've already finished. Just plug it into a solver and wait for your Nobel prize / acquisition by Google.

## An Example

An example of an integer linear program is as follows:

**objective function**:

$F = \sum_{i=0}^{N}x_i V_i$

**constraints**:

$\sum_{i=0}^{N}x_i C_i \le 100$

$\sum_{i=0}^{N}x_i \le 10$

This describes the following problem:
There are $N$ different items in a shop, each with an associated cost $C_i$ and value $V_i$. We indicate whether we buy each item or not with a binary variable $x_i$, so we have 100 variables in total. We want to maximise the total value of the items we buy. However:
1. we can only spend 100 pounds
2. we can only buy up to 10 items

## PuLP

PuLP is a linear programming library in Python. It allows you to write down an objective function and constraints in a very intuitive way and instantly solve them. Let's see an example with the above problem.

In [2]:
import pulp
import numpy as np

# random fake data for costs and values
costs = np.random.uniform(low=5, high=20, size=100)
values = costs * np.random.uniform(low=0.9, high=1.1, size=100)

model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
decisions = [pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
             for i in range(100)]

# PuLP has a slightly weird syntax, but it's great. This is how to add the objective function:
model += sum(decisions[i] * values[i] for i in range(100)), "Objective"

# and here are the constraints
model += sum(decisions[i] * costs[i] for i in range(100)) <= 100  # total cost
model += sum(decisions) <= 10  # total items

model.solve()

# print results
for i in range(100):
    if decisions[i].value() == 1:
        print(i, costs[i], values[i])

6 8.341428313257937 9.100268198102812
35 13.600960205648033 14.934066372309184
39 13.69133612534416 15.0583982150792
42 11.376117487345171 12.454501311848414
54 5.5834232434849005 6.126090499083039
60 11.948450673927253 12.930444994011147
70 17.81557727312878 19.376777977556543
83 5.223790414247662 5.58954351922558
92 12.377265233550922 13.613949258480199


## Picking a Fantasy Premier League Team

We are now going to apply the same techniques to solve a simplified team selection task. The simplifications we will make are as follows:

* No substitutes
* No transfers
* Assume we have perfect forecasts

We can address some of these simplifications later. We need to add the following features:

* Formation constraints: max and min number of players in each position
* Team constraints: max 3 players from each club
* Captaincy: extra decision array for picking a captain, with associated cost function contribution and constraints.

Let's write down the objective and constraints for this new problem.

$F = \sum_{i=0}^{N}(x_i + y_i) * V_i$

Where $x_i$ is a binary decision indicating whether player $i$ is in the starting eleven, and $y_i$ indicates whether he is the captain. Here $V_i$ is the expected score for player $i$.

constraints:
1. $\sum_{i=0}^{N}x_i * C_i \le 100$
2. $\sum_{i=0}^{N}x_i \le 11$
3. $\sum_{i=0}^{N}y_i = 1$
4. $\sum_{j \in G}x_i = 1$
5. $3 \le \sum_{j \in D}x_j \le 5$
6. $3 \le \sum_{j \in M}x_j \le 5$
7. $1 \le \sum_{j \in F}x_j \le 3$

Where $G$, $D$, $M$ and $F$ are the sets of all goalkeepers, defenders, midfielders and forwards respectively.

8. For each team $k$: $\sum_{j \in T_k}x_j \le 3$, where $T_k$ is the set of all players on team $k$. I.e. you cannot select more than 3 players from each team.

9. For each player $i$: $x_i - y_i = 0$

This last set of constraints is a slightly awkward way of saying that the captain must also be on the team -- either $x_i$ and $y_i$ are both zero or both one. We apply this constraint to each player individual, which leads to several hundred constraints. This is a little messy, but it doesn't cause problems.

If the above is a little confusing, hopefully it is clearer written as code:

In [3]:
def select_team(expected_scores, prices, positions, clubs):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]

    # objective function:
    model += sum((captain_decisions[i] + decisions[i]) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum(decisions[i] * prices[i] for i in range(num_players)) <= 100  # total cost
    model += sum(decisions) == 11  # total team size

    # position constraints
    # 1 goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 1) == 1
    # 3-5 defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) <= 5
    # 3-5 midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) <= 5
    # 1-3 attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) <= 3

    # club constraint
    for club_id in np.unique(clubs):
        model += sum(decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

    model += sum(captain_decisions) == 1  # 1 captain
    
    for i in range(num_players):  # captain must also be on team
        model += (decisions[i] - captain_decisions[i]) >= 0

    model.solve()
    return decisions, captain_decisions

In [4]:
expected_scores = np.random.uniform(low=5, high=20, size=100)
prices = expected_scores * np.random.uniform(low=0.9, high=1.1, size=100)
positions = np.random.randint(1, 5, size=100)
clubs = np.random.randint(0, 20, size=100)
decisions, captain_decisions = select_team(expected_scores, prices, positions, clubs)

In [5]:
# print results
for i in range(100):
    if decisions[i].value() != 0:
        print(i,  expected_scores[i], prices[i])

print("Captain:")
# print results
for i in range(100):
    if captain_decisions[i].value() == 1:
        print(i, expected_scores[i], prices[i])


9 19.406470556707163 17.94236009164756
11 5.359729723221568 5.2283965828963845
13 6.877626149236996 6.33329927400512
17 5.78826156620296 5.212946279060511
43 17.460967462205332 15.8656021478448
45 8.865567085015249 8.243248816622955
50 9.099948237470958 8.42034937449553
56 8.535987829149041 7.906212899047144
60 9.65278204142739 8.839921233730426
63 7.475365623821652 6.824007186371284
64 9.867420753317024 9.028363634171647
Captain:
9 19.406470556707163 17.94236009164756


These fake players don't mean anything -- let's try this with real data.

The FPL app allows you to access all player data through a number of API endpoints. For simplicity, we are using a ready-made data file from the archive collected by [vaastav](https://github.com/vaastav/Fantasy-Premier-League).

In [6]:
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/players_raw.csv"
)
df[["web_name", "total_points", "team_code", "element_type", "now_cost"]].head()

,web_name,total_points,team_code,element_type,now_cost
0,Mustafi,80,3,2,55
1,Bellerín,60,3,2,55
2,Kolasinac,81,3,2,55
3,Maitland-Niles,34,3,2,50
4,Sokratis,64,3,2,50


We'll now solve a semi-useful problem:
* Use the total points from the last season as a basic forecast of the total points for this season.
* Pick the optimal team assuming this forecast is correct and transfers are not allowed.

This should give a decent first team for the new season.

In [7]:
expected_scores = df["total_points"]  # total points from last season
prices = df["now_cost"] / 10
positions = df["element_type"]
clubs = df["team_code"]
# so we can read the results
names = df["web_name"]
decisions, captain_decisions = select_team(expected_scores.values, prices.values, positions.values, clubs.values)

In [8]:
for i in range(df.shape[0]):
    if decisions[i].value() != 0:
        display(Markdown("**{}** Points = {}, Price = {}".format(names[i], expected_scores[i], prices[i])))

for i in range(df.shape[0]):
    if captain_decisions[i].value() == 1:
        (display(Markdown("**CAPTAIN: {}** Points = {}, Price = {}".format(names[i], expected_scores[i], prices[i]))))


**Aubameyang** Points = 205, Price = 11.0

**David Luiz** Points = 164, Price = 6.0

**Pickford** Points = 161, Price = 5.5

**Sigurdsson** Points = 182, Price = 8.0

**Robertson** Points = 213, Price = 7.0

...

**Laporte** Points = 177, Price = 6.5

**Agüero** Points = 201, Price = 12.0

**Sterling** Points = 234, Price = 12.0

**Jiménez** Points = 181, Price = 7.5

**CAPTAIN: Salah** Points = 259, Price = 12.5

In [9]:
prices

0       5.5
1       5.5
2       5.5
3       5.0
4       5.0
5       5.0
6       5.0
7       4.5
8       4.5
9       4.5
10     11.0
11      9.5
12      4.5
13      5.0
14      7.5
15      7.0
16      6.0
17      5.5
18      5.0
19      4.5
20      4.5
21      4.5
22      4.5
23      5.5
24      9.5
25      5.5
26      5.0
27      4.5
28      4.5
29      4.5
       ... 
481     5.0
482     4.5
483     4.5
484     6.5
485     4.5
486     4.5
487     7.5
488     6.0
489     6.0
490     5.5
491     5.0
492     5.0
493     5.0
494     4.5
495     4.5
496     4.0
497     7.5
498     6.5
499     5.0
500     4.5
501     4.0
502     5.5
503     5.5
504     5.0
505     5.0
506     5.0
507     5.0
508     4.5
509     5.0
510     6.0
Name: now_cost, Length: 511, dtype: float64

In [10]:
decisions, captain_decisions = select_team(expected_scores.values, prices.values, positions.values, clubs.values)

Sadly we can't pick this team, since we also need to pay for subs. We can fudge subs into the model by treating them like regular players whose scores are reduced by a fudge factor.

In [16]:
def select_team(expected_scores, prices, positions, clubs, total_budget=100, sub_factor=0.2):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]


    # objective function:
    model += sum((captain_decisions[i] + decisions[i] + sub_decisions[i]*sub_factor) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum((decisions[i] + sub_decisions[i]) * prices[i] for i in range(num_players)) <= total_budget  # total cost

    # position constraints
    # 1 starting goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 1) == 1
    # 2 total goalkeepers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 1) == 2

    # 3-5 starting defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) <= 5
    # 5 total defenders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 2) == 5

    # 3-5 starting midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) <= 5
    # 5 total midfielders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 3) == 5

    # 1-3 starting attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) <= 3
    # 3 total attackers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 4) == 3

    # club constraint
    for club_id in np.unique(clubs):
        model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

    model += sum(decisions) == 11  # total team size
    model += sum(captain_decisions) == 1  # 1 captain
    
    for i in range(num_players):  
        model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
        model += (decisions[i] + sub_decisions[i]) <= 1  # subs must not be on team

    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return decisions, captain_decisions, sub_decisions

In [17]:
decisions, captain_decisions, sub_decisions = select_team(expected_scores.values, prices.values, positions.values, clubs.values)
# print results
for i in range(df.shape[0]):
    if decisions[i].value() != 0:
        display(Markdown("**{}** Points = {}, Price = {}".format(names[i], expected_scores[i], prices[i])))
print()
print("Subs:")
# print results
for i in range(df.shape[0]):
    if sub_decisions[i].value() == 1:
        display(Markdown("**{}** Points = {}, Price = {}".format(names[i], expected_scores[i], prices[i])))

print()
print("Captain:")
# print results
for i in range(df.shape[0]):
    if captain_decisions[i].value() == 1:
        display(Markdown("**{}** Points = {}, Price = {}".format(names[i], expected_scores[i], prices[i])))


Total expected score = 2383.9999999999995


**Fraser** Points = 181, Price = 7.5

**David Luiz** Points = 164, Price = 6.0

**Milivojevic** Points = 166, Price = 7.0

**Digne** Points = 158, Price = 6.0

...

**Kanté** Points = 116, Price = 5.0

**Long** Points = 66, Price = 5.0

**Fabianski** Points = 143, Price = 5.0


Captain:


**Salah** Points = 259, Price = 12.5

If we think the subs are too expensive, we can play with the sub factor:

In [13]:
decisions, captain_decisions, sub_decisions = select_team(expected_scores.values, prices.values,
                                                          positions.values, clubs.values,
                                                          sub_factor=0.2)
# print results
for i in range(df.shape[0]):
    if decisions[i].value() != 0:
        print(names[i], expected_scores[i], prices[i])
print()
print("Subs:")
# print results
for i in range(df.shape[0]):
    if sub_decisions[i].value() == 1:
        print(names[i], expected_scores[i], prices[i])

print()
print("Captain:")
# print results
for i in range(df.shape[0]):
    if captain_decisions[i].value() == 1:
        print(names[i], expected_scores[i], prices[i])


Total expected score = 2350.5
Fraser 181 7.5
Azpilicueta 158 6.0
David Luiz 164 6.0
Milivojevic 166 7.0
Sigurdsson 182 8.0
Robertson 213 7.0
van Dijk 208 6.5
Salah 259 12.5
Laporte 177 6.5
Ederson 169 6.0
Jiménez 181 7.5

Subs:
Nketiah 10 4.5
Kanté 116 5.0
Long 66 5.0
Fabianski 143 5.0

Captain:
Salah 259 12.5
